In [9]:
import pandas as pd

# dados que vou usar para a agregação
ibge = pd.read_csv('gs://trabalho-final-pdm/silver/ibge_goiania.csv', storage_options={"token": "google_default"})

# 2016
tse = pd.read_csv('gs://trabalho-final-pdm/silver/tse_bairros_2024.csv', storage_options={"token": "google_default"})


zonas = pd.read_csv('gs://trabalho-final-pdm/bronze/zonas_goiania', storage_options={"token": "google_default"})

In [10]:
ibge.head()

,Nome_do_subdistrito,Nome_do_bairro,Situacao_setor_x,Proporcao_negros_pardos_indigenas,Proporcao_feminino,Proporcao_18_39,V001,V002,V005,V007,V009,V011
0,U.T.P. CENTRAL,GOIANIA (TODOS OS SETORES),1,0.441964,0.543155,0.245536,276.0,642.0,"2432,47","2543,03","1588,1","2084,38"
1,U.T.P. CENTRAL,GOIANIA (TODOS OS SETORES),1,0.352713,0.608527,0.178295,225.0,491.0,"2198,14","2389,29","1512,69","2008,52"
2,U.T.P. CENTRAL,GOIANIA (TODOS OS SETORES),1,0.567241,0.546552,0.243103,250.0,584.0,"1787,44","1885,48","1233,05","1513,68"
3,U.T.P. CENTRAL,GOIANIA (TODOS OS SETORES),1,0.471698,0.574124,0.196765,217.0,363.0,"2209,32","2282,96","1730,26","2051,34"
4,U.T.P. CENTRAL,GOIANIA (TODOS OS SETORES),1,0.439799,0.555184,0.204013,214.0,571.0,"3044,29","3257,39","1871,55","2607,5"


In [11]:
tse.head()

,ANO_ELEICAO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_TIPO_ELEICAO,NR_TURNO,DS_CARGO,NM_VOTAVEL,NR_VOTAVEL,QT_VOTOS,NM_LOCAL_VOTACAO,DS_LOCAL_VOTACAO_ENDERECO,LOCAL DE VOTACAO,ENDEREÇO,BAIRRO
0,2024,GOIANIA,2,35,2,1,VEREADOR,NATANIEL DE SENA SOARES,25190,2,ESCOLA MUNICIPAL PROFA EDNA DE ROURE,RUA DO MERCADO N 89,['CENTRO DE ENSINO EM PERIODO INTEGRAL AECIO O...,"['RUA CARIJOS COM RUA DO PRATA', 'AV SOL NASCE...","['SETOR URIAS MAGALHAES', 'JARDIM NOVA ESPERAN..."
1,2024,GOIANIA,147,399,2,1,VEREADOR,FERNANDO GOMES DE SOUSA,40555,1,FACULDADE NOROESTE - FAN (PROJETO DIDATICO),RUA CREPUSCULO QD. 17 LTS. 168 A 179,"['COLEGIO DESAFIO', 'COLEGIO ESTADUAL PROFESSO...","['AV MANGALO QD 55 LT 06', 'AV. FERNAO DIAS PA...","['SETOR MORADA DO SOL', 'CAPUAVA', 'VILA MULTI..."
2,2024,GOIANIA,146,171,2,1,VEREADOR,MARIANA SILVA MARTINS GIDRAO,44100,1,ESCOLA MUNICIPAL LAURICIO PEDRO RASMUSSEM,RUA L-3 QD.22 N.250,"['CEPMGO WALDEMAR MUNDIM', 'CEPMGO JARDIM GUAN...","['RUA R-40 QD40 LT07', 'ALAMEDA MINAS GERAIS Q...","['VILA ITATIAIA', 'JARDIM GUANABARA', 'LESTE V..."
3,2024,GOIANIA,2,92,2,1,VEREADOR,NATANIEL DE SENA SOARES,25190,1,CENTRO DE ENSINO EM PERIODO INTEGRAL CASTRO ALVES,AV DA DIVISA N 76,['CENTRO DE ENSINO EM PERIODO INTEGRAL AECIO O...,"['RUA CARIJOS COM RUA DO PRATA', 'AV SOL NASCE...","['SETOR URIAS MAGALHAES', 'JARDIM NOVA ESPERAN..."
4,2024,GOIANIA,146,239,2,1,VEREADOR,NATANIEL DE SENA SOARES,25190,4,COLEGIO ESTADUAL JORNALISTA LUIZ GONZAGA CONTART,RUA GB-04 ESQ C/ GB-05,"['CEPMGO WALDEMAR MUNDIM', 'CEPMGO JARDIM GUAN...","['RUA R-40 QD40 LT07', 'ALAMEDA MINAS GERAIS Q...","['VILA ITATIAIA', 'JARDIM GUANABARA', 'LESTE V..."


In [12]:
zonas.head()

,ZONA,LOCAL DE VOTACAO,ENDEREÇO,BAIRRO,total de seções,total de eleitores
0,1,Colegio Agostiniano Nossa Senhora de Fatima,"Av K, 108 - Unidade I",Setor Aeroporto,19,7.291
1,147,Colegio Desafio,Av Mangalo Qd 55 Lt 06,Setor Morada do Sol,18,6.944
2,136,Colegio Estadual Jardim Vila Boa,Rua Paola Ney Qd. 59,Jardim Vila Boa,20,6.875
3,127,Escola Sesi Jardim Planalto,Praca Itapua Qd 30 150,Jardim Planalto,20,6.524
4,127,Colegio Estadual Polivalente Profº Goiany Prates,Rua C-68 Qd 115 336,Vilas dos Alpes,17,6.389


In [13]:
tse.shape

(326721, 15)

In [14]:
# ultimas transformações em IBGE para a agregação
# Vamos ter que entender essa tabela como uma amostra, porque eu filtrei teoricamente pela cidade, então, 
# se o censo passar por todas as casas, no minimo deveria ter 1 milhão de linhas esse dataset

# Converter as colunas numéricas para o tipo float
ibge["V005"] = ibge["V005"].str.replace(",", ".").astype(float)
ibge["V009"] = ibge["V009"].str.replace(",", ".").astype(float)

# De salarios deixei apenas o que tenho certeza
# Acho que esses dados representam o setor cencitário, mas queremos so valores por setor

ibge = ibge[['Nome_do_subdistrito', 'Situacao_setor_x', 'Proporcao_negros_pardos_indigenas','Proporcao_feminino','Proporcao_18_39', 'V005', 'V009']]

# Certificar que todas as colunas numéricas estão em formato correto
ibge["V005"] = pd.to_numeric(ibge["V005"], errors="coerce")
ibge["V009"] = pd.to_numeric(ibge["V009"], errors="coerce")
ibge[["Proporcao_negros_pardos_indigenas", "Proporcao_feminino", "Proporcao_18_39"]] = ibge[["Proporcao_negros_pardos_indigenas",
                                                                                             "Proporcao_feminino",
                                                                                             "Proporcao_18_39"]].astype(float)

# Agregar os dados por Nome_do_subdistrito
ibge_agrupado = ibge.groupby("Nome_do_subdistrito").agg({
    "Situacao_setor_x": lambda x: x.mode()[0],  # Pega o valor mais comum
    "Proporcao_negros_pardos_indigenas": "mean",
    "Proporcao_feminino": "mean", 
    "Proporcao_18_39": "mean",
    "V005": "mean",  # Média dos valores
    "V009": "mean",  # Média dos valores
}).reset_index()

# Ver o resultado
ibge_agrupado

,Nome_do_subdistrito,Situacao_setor_x,Proporcao_negros_pardos_indigenas,Proporcao_feminino,Proporcao_18_39,V005,V009
0,U.T.P. AEROPORTO,1,0.298498,0.551503,0.198365,3679.232000,2220.962000
1,U.T.P. AEROPORTO INTERNACIONAL SANTA GENOVEVA,1,0.630435,0.478261,0.108696,3435.710000,1542.860000
2,U.T.P. AEROVIARIOS,1,0.527839,0.526754,0.223338,1243.974737,806.283158
3,U.T.P. ALTO DA GLORIA E REDENCAO,1,0.399619,0.547379,0.238300,3067.470400,2045.876400
4,U.T.P. AUTODROMO OU PARQUE LOZANDES,1,0.208793,0.518386,0.188219,8360.048000,4542.501000
...,...,...,...,...,...,...,...
59,U.T.P. VILA PEDROSO,1,0.622421,0.511540,0.241297,880.640800,577.509600
60,U.T.P. VILA REGINA/PARQUE INDUSTRIAL PAULISTA/...,1,0.551199,0.508213,0.250428,1199.879286,750.282143
61,U.T.P. VILA RIZZO,1,0.626451,0.500592,0.258921,874.826000,597.820667
62,U.T.P. ZONA RURAL,1,0.597443,0.483680,0.228838,860.635952,536.335714


In [7]:
!pip install scikit-learn

In [15]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Função para imputar bairros usando TF-IDF
def imputar_bairros(tse_texts, zonas_texts, zonas_df, threshold=0.7):
    """
    Parâmetros:
    tse_texts: Lista de textos da tabela tse (NM_LOCAL_VOTACAO)
    zonas_texts: Lista de textos da tabela zonas (LOCAL DE VOTACAO)
    zonas_df: DataFrame com dados da tabela zonas
    threshold: Valor mínimo de similaridade para imputação
    """
    vectorizer = TfidfVectorizer()
    vectorizer.fit(zonas_texts)

    # Vetorizar os textos
    tse_vetorizado = vectorizer.transform(tse_texts)
    zonas_vetorizado = vectorizer.transform(zonas_texts)

    # Calcular similaridade
    similaridades = cosine_similarity(tse_vetorizado, zonas_vetorizado)

    # Criar lista com bairros imputados
    bairros_imputados = []
    for i, similaridades_linha in enumerate(similaridades):
        max_similaridade = similaridades_linha.max()
        if max_similaridade >= threshold:
            idx = similaridades_linha.argmax()
            bairro = zonas_df.iloc[idx]["BAIRRO"]
        else:
            bairro = None  # Sem similaridade suficiente
        bairros_imputados.append(bairro)

    return bairros_imputados

# Preparar os textos
tse_texts = tse["NM_LOCAL_VOTACAO"].fillna("").tolist()
zonas_texts = zonas["LOCAL DE VOTACAO"].fillna("").tolist()

# Imputar os bairros
bairros_imputados = imputar_bairros(tse_texts, zonas_texts, zonas, threshold=0.7)

# Adicionar a nova coluna 'bairros_imputados' no DataFrame tse
tse["bairros_imputados"] = bairros_imputados

# Exibir o resultado final
tse


,ANO_ELEICAO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_TIPO_ELEICAO,NR_TURNO,DS_CARGO,NM_VOTAVEL,NR_VOTAVEL,QT_VOTOS,NM_LOCAL_VOTACAO,DS_LOCAL_VOTACAO_ENDERECO,LOCAL DE VOTACAO,ENDEREÇO,BAIRRO,bairros_imputados
0,2024,GOIANIA,2,35,2,1,VEREADOR,NATANIEL DE SENA SOARES,25190,2,ESCOLA MUNICIPAL PROFA EDNA DE ROURE,RUA DO MERCADO N 89,['CENTRO DE ENSINO EM PERIODO INTEGRAL AECIO O...,"['RUA CARIJOS COM RUA DO PRATA', 'AV SOL NASCE...","['SETOR URIAS MAGALHAES', 'JARDIM NOVA ESPERAN...",Vila Irany
1,2024,GOIANIA,147,399,2,1,VEREADOR,FERNANDO GOMES DE SOUSA,40555,1,FACULDADE NOROESTE - FAN (PROJETO DIDATICO),RUA CREPUSCULO QD. 17 LTS. 168 A 179,"['COLEGIO DESAFIO', 'COLEGIO ESTADUAL PROFESSO...","['AV MANGALO QD 55 LT 06', 'AV. FERNAO DIAS PA...","['SETOR MORADA DO SOL', 'CAPUAVA', 'VILA MULTI...",Setor Morada do Sol
2,2024,GOIANIA,146,171,2,1,VEREADOR,MARIANA SILVA MARTINS GIDRAO,44100,1,ESCOLA MUNICIPAL LAURICIO PEDRO RASMUSSEM,RUA L-3 QD.22 N.250,"['CEPMGO WALDEMAR MUNDIM', 'CEPMGO JARDIM GUAN...","['RUA R-40 QD40 LT07', 'ALAMEDA MINAS GERAIS Q...","['VILA ITATIAIA', 'JARDIM GUANABARA', 'LESTE V...",Bairro Feliz
3,2024,GOIANIA,2,92,2,1,VEREADOR,NATANIEL DE SENA SOARES,25190,1,CENTRO DE ENSINO EM PERIODO INTEGRAL CASTRO ALVES,AV DA DIVISA N 76,['CENTRO DE ENSINO EM PERIODO INTEGRAL AECIO O...,"['RUA CARIJOS COM RUA DO PRATA', 'AV SOL NASCE...","['SETOR URIAS MAGALHAES', 'JARDIM NOVA ESPERAN...",Vila Santa Helena
4,2024,GOIANIA,146,239,2,1,VEREADOR,NATANIEL DE SENA SOARES,25190,4,COLEGIO ESTADUAL JORNALISTA LUIZ GONZAGA CONTART,RUA GB-04 ESQ C/ GB-05,"['CEPMGO WALDEMAR MUNDIM', 'CEPMGO JARDIM GUAN...","['RUA R-40 QD40 LT07', 'ALAMEDA MINAS GERAIS Q...","['VILA ITATIAIA', 'JARDIM GUANABARA', 'LESTE V...",Jardim Guanabara II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326716,2024,GOIANIA,127,161,2,1,VEREADOR,MARCOS ANTONIO DA SILVA,25456,1,CENTRO MUNICIPAL DE ENSINO INFANTIL JOAO PAULO I,RUA C-169 QD. 415 N 1595,"['ESCOLA SESI JARDIM PLANALTO', 'COLEGIO ESTAD...","['PRACA ITAPUA QD 30 150', 'RUA C-68 QD 115 33...","['JARDIM PLANALTO', 'VILAS DOS ALPES', 'JARDIM...",Jardim América
326717,2024,GOIANIA,147,31,2,1,VEREADOR,MARCOS ANTONIO DA SILVA,25456,1,ESCOLA MUNICIPAL PEDRO GOMES DE MENEZES,RUA MANOEL SILVA QD 08 LT 01 VILA REGINA,"['COLEGIO DESAFIO', 'COLEGIO ESTADUAL PROFESSO...","['AV MANGALO QD 55 LT 06', 'AV. FERNAO DIAS PA...","['SETOR MORADA DO SOL', 'CAPUAVA', 'VILA MULTI...",Vila Regina
326718,2024,GOIANIA,2,367,2,1,VEREADOR,MARCOS ANTONIO DA SILVA,25456,2,CENTRO DE ENSINO EM PERIODO INTEGRAL AECIO OLI...,RUA CARIJOS ESQ C/ RUA DO PRATA S/N,['CENTRO DE ENSINO EM PERIODO INTEGRAL AECIO O...,"['RUA CARIJOS COM RUA DO PRATA', 'AV SOL NASCE...","['SETOR URIAS MAGALHAES', 'JARDIM NOVA ESPERAN...",Setor Urias Magalhães
326719,2024,GOIANIA,136,362,2,1,VEREADOR,MARCOS ANTONIO DA SILVA,25456,1,ESCOLA GOVERNADOR JOAQUIM SOBROSA - LIONS,R.ESTRADA DUQUE DE CAXIAS QD 3 LT 48/49,"['COLEGIO ESTADUAL JARDIM VILA BOA', 'COLEGIO ...","['RUA PAOLA NEY QD. 59', 'RUA B-16 101', 'RUA ...","['JARDIM VILA BOA', 'VILA NOVO HORIZONTE', 'SE...",Setor Cristina


In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Função para encontrar os melhores matches com um threshold
def match_tfidf(tse_texts, ibge_texts, ibge_df, threshold=0.5):
    """
    Parâmetros:
    tse_texts: Lista de textos da tabela tse
    ibge_texts: Lista de textos da tabela ibge
    ibge_df: DataFrame com dados do IBGE
    threshold: Limite mínimo de similaridade para considerar um match
    """
    vectorizer = TfidfVectorizer()
    vectorizer.fit(ibge_texts)

    # Vetorizar os textos
    tse_vetorizado = vectorizer.transform(tse_texts)
    ibge_vetorizado = vectorizer.transform(ibge_texts)

    # Calcular a similaridade de coseno
    similaridades = cosine_similarity(tse_vetorizado, ibge_vetorizado)

    # Encontrar o índice com maior similaridade
    melhores_matches = []
    for i, similaridades_linha in enumerate(similaridades):
        max_similaridade = similaridades_linha.max()
        if max_similaridade >= threshold:
            idx = similaridades_linha.argmax()
            match = ibge_df.iloc[idx].to_dict()
            match['similaridade'] = max_similaridade  # Adicionar a similaridade
        else:
            # Se a similaridade for menor que o threshold, criar linha vazia
            match = {col: None for col in ibge_df.columns}
            match['similaridade'] = max_similaridade
        melhores_matches.append(match)
    
    return melhores_matches

# Criar nova coluna combinada em tse (DS_LOCAL_VOTACAO_ENDERECO + bairros_imputados)
tse["combinado"] = tse["DS_LOCAL_VOTACAO_ENDERECO"].fillna("") + " " + tse["bairros_imputados"].fillna("")

# Criar nova coluna combinada em ibge (Nome_do_subdistrito, ou ajustar conforme necessário)
ibge["combinado"] = ibge["Nome_do_subdistrito"].fillna("")

# Preparar os textos para TF-IDF
tse_texts = tse["combinado"].tolist()
ibge_texts = ibge["combinado"].tolist()

# Definir o threshold mínimo de similaridade
threshold_similaridade = 0.5

# Realizar os matches com threshold
matches = match_tfidf(tse_texts, ibge_texts, ibge, threshold=threshold_similaridade)

# Combinar os DataFrames
tse_matches = pd.DataFrame(matches)
tse_unificado = pd.concat([tse.reset_index(drop=True), tse_matches.reset_index(drop=True)], axis=1)

# Exibir o resultado final
tse_unificado


,ANO_ELEICAO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_TIPO_ELEICAO,NR_TURNO,DS_CARGO,NM_VOTAVEL,NR_VOTAVEL,QT_VOTOS,...,combinado,Nome_do_subdistrito,Situacao_setor_x,Proporcao_negros_pardos_indigenas,Proporcao_feminino,Proporcao_18_39,V005,V009,combinado,similaridade
0,2024,GOIANIA,2,35,2,1,VEREADOR,NATANIEL DE SENA SOARES,25190,2,...,RUA DO MERCADO N 89 Vila Irany,None,NaN,NaN,NaN,NaN,NaN,NaN,None,0.441343
1,2024,GOIANIA,147,399,2,1,VEREADOR,FERNANDO GOMES DE SOUSA,40555,1,...,RUA CREPUSCULO QD. 17 LTS. 168 A 179 Setor Mor...,U.T.P. CEASA/ALDEIA DO VALE,1.0,0.158405,0.513390,0.156695,10528.32,6537.06,U.T.P. CEASA/ALDEIA DO VALE,0.500000
2,2024,GOIANIA,146,171,2,1,VEREADOR,MARIANA SILVA MARTINS GIDRAO,44100,1,...,RUA L-3 QD.22 N.250 Bairro Feliz,U.T.P. BAIRRO FELIZ,1.0,0.389021,0.527446,0.193317,2964.65,1716.74,U.T.P. BAIRRO FELIZ,1.000000
3,2024,GOIANIA,2,92,2,1,VEREADOR,NATANIEL DE SENA SOARES,25190,1,...,AV DA DIVISA N 76 Vila Santa Helena,None,NaN,NaN,NaN,NaN,NaN,NaN,None,0.409613
4,2024,GOIANIA,146,239,2,1,VEREADOR,NATANIEL DE SENA SOARES,25190,4,...,RUA GB-04 ESQ C/ GB-05 Jardim Guanabara II,U.T.P. JARDIM GUANABARA,1.0,0.533489,0.525701,0.214953,1831.23,989.61,U.T.P. JARDIM GUANABARA,0.646770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326716,2024,GOIANIA,127,161,2,1,VEREADOR,MARCOS ANTONIO DA SILVA,25456,1,...,RUA C-169 QD. 415 N 1595 Jardim América,None,NaN,NaN,NaN,NaN,NaN,NaN,None,0.492037
326717,2024,GOIANIA,147,31,2,1,VEREADOR,MARCOS ANTONIO DA SILVA,25456,1,...,RUA MANOEL SILVA QD 08 LT 01 VILA REGINA Vila...,U.T.P. VILA REGINA/PARQUE INDUSTRIAL PAULISTA/...,1.0,0.568110,0.525871,0.241816,806.89,528.78,U.T.P. VILA REGINA/PARQUE INDUSTRIAL PAULISTA/...,0.514103
326718,2024,GOIANIA,2,367,2,1,VEREADOR,MARCOS ANTONIO DA SILVA,25456,2,...,RUA CARIJOS ESQ C/ RUA DO PRATA S/N Setor Uria...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,0.408829
326719,2024,GOIANIA,136,362,2,1,VEREADOR,MARCOS ANTONIO DA SILVA,25456,1,...,R.ESTRADA DUQUE DE CAXIAS QD 3 LT 48/49 Setor ...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,0.487514


In [17]:
tse_unificado.isnull().sum()

ANO_ELEICAO                               0
NM_MUNICIPIO                              0
NR_ZONA                                   0
NR_SECAO                                  0
CD_TIPO_ELEICAO                           0
NR_TURNO                                  0
DS_CARGO                                  0
NM_VOTAVEL                                0
NR_VOTAVEL                                0
QT_VOTOS                                  0
NM_LOCAL_VOTACAO                          0
DS_LOCAL_VOTACAO_ENDERECO                 0
LOCAL DE VOTACAO                          0
ENDEREÇO                                  0
BAIRRO                                    0
bairros_imputados                     25426
combinado                                 0
Nome_do_subdistrito                  102142
Situacao_setor_x                     102142
Proporcao_negros_pardos_indigenas    102142
Proporcao_feminino                   102142
Proporcao_18_39                      102142
V005                            

In [18]:
tse_unificado.shape

(326721, 26)

In [19]:
# seleção das features boas

# seleção das features boas

tse_unificado = tse_unificado[['ANO_ELEICAO', 
                               'NM_MUNICIPIO',
                               'NR_ZONA','NR_SECAO', 
                               'CD_TIPO_ELEICAO', 
                               'NR_TURNO', 'DS_CARGO',
                               'NM_VOTAVEL',
                               'NR_VOTAVEL',
                               'QT_VOTOS',
                               'NM_LOCAL_VOTACAO',
                               'Nome_do_subdistrito', 
                               'Situacao_setor_x',
                               'V005',
                               'V009',
                               'Proporcao_negros_pardos_indigenas',
                               'Proporcao_feminino',
                               'Proporcao_18_39' ]]

In [20]:
# Salvar DataFrame diretamente no GCS
gcs_path = "gs://trabalho-final-pdm/silver/agregados_2024.csv"

# Escrever para o Cloud Storage usando storage_options
tse_unificado.to_csv(gcs_path, index=False, storage_options={"token": "google_default"})
